tempo:
- ammazzare 

- ingannare 

- pioggia 

- mele 

- spazio

Cercare le parole nelle frasi del dataset, restiruire le frasi che le cotnengono.
pulire le frasi ottenute, per restituire "tempo" da "ammazzare il tempo" (stopwords e initialwords)
restituirer la parola più frequente tra quelle restituite

In [185]:
from nltk.corpus import stopwords
print(stopwords.words('italian'))

['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl', 'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal', 'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle', 'di', 'del', 'dello', 'dei', 'degli', 'dell', 'degl', 'della', 'delle', 'in', 'nel', 'nello', 'nei', 'negli', 'nell', 'negl', 'nella', 'nelle', 'su', 'sul', 'sullo', 'sui', 'sugli', 'sull', 'sugl', 'sulla', 'sulle', 'per', 'tra', 'contro', 'io', 'tu', 'lui', 'lei', 'noi', 'voi', 'loro', 'mio', 'mia', 'miei', 'mie', 'tuo', 'tua', 'tuoi', 'tue', 'suo', 'sua', 'suoi', 'sue', 'nostro', 'nostra', 'nostri', 'nostre', 'vostro', 'vostra', 'vostri', 'vostre', 'mi', 'ti', 'ci', 'vi', 'lo', 'la', 'li', 'le', 'gli', 'ne', 'il', 'un', 'uno', 'una', 'ma', 'ed', 'se', 'perché', 'anche', 'come', 'dov', 'dove', 'che', 'chi', 'cui', 'non', 'più', 'quale', 'quanto', 'quanti', 'quanta', 'quante', 'quello', 'quelli', 'quella', 'quelle', 'questo', 'questi', 'questa', 'queste', 'si', 'tutto', 'tutti', 'a', 'c', 'e', 'i', 'l', 'o', 'ho', 'hai', 'ha', 'ab

In [186]:
WORDS_1 = ['ammazzare', 'ingannare', 'pioggia', 'mele', 'spazio']
WORDS_2 = ['bicchiere', 'mulino', 'gocce', 'gola', 'buco']

In [187]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

In [188]:
def check_dataset(path, words):
    phrases = []    
    with open(path, 'r') as file:
        for line in file:
            for word in words:
                if word.lower() in line.lower():  # Salviamo le frasi che contengono le parole in input
                    line = line.lower()
                    line = line.replace(word, '')
                    phrases.append(remove_stop_words_ita(line)) # Rimuoviamo dalla frase le stopword e le parole in input
    return phrases

In [189]:
def most_frequent_words(phrases):
    words = {}
    for phrase in phrases:
        for word in phrase:
            if word in words:
                words[word] += 1
            else:
                words[word] = 1
    return words

def get_most_frequent_word(words):
    max_word = ''
    max_count = 0
    for word in words:
        if words[word] > max_count:
            max_word = word
            max_count = words[word]
    return max_word

In [193]:
ph = check_dataset('..\ghigliottina.txt', WORDS_2)
print(ph)
dict = most_frequent_words(ph)
print(dict)
answer = get_most_frequent_word(dict)
print(f"\nLa sesta parola è: {answer}")

[['acqua'], ['staffa'], ['fare', 'acqua'], ['re', 'pollice'], ['andare', 'buca', '/'], ['avere', 'cuore'], ['avere', 'carte', 're'], ['avere', 'groppo'], ['acqua'], ['nero'], ['genio', 'sretezza'], ['acqua'], ['mandare', 'acquaio'], ['mentire'], ['peccato'], ['perdersi', 'd', 'acqua'], ['pietra', 'anre', '/', 'pietra', 'd', 'angolo'], ['portare', 'acqua', 'proprio'], ['prendere'], ['somigliarsi', 'due', 'd', 'acqua'], ['strappo', 're'], ['tempesta', 'd', 'acqua'], ['tirare', 'acqua', 'proprio']]
{'acqua': 9, 'staffa': 1, 'fare': 1, 're': 3, 'pollice': 1, 'andare': 1, 'buca': 1, '/': 2, 'avere': 3, 'cuore': 1, 'carte': 1, 'groppo': 1, 'nero': 1, 'genio': 1, 'sretezza': 1, 'mandare': 1, 'acquaio': 1, 'mentire': 1, 'peccato': 1, 'perdersi': 1, 'd': 4, 'pietra': 2, 'anre': 1, 'angolo': 1, 'portare': 1, 'proprio': 2, 'prendere': 1, 'somigliarsi': 1, 'due': 1, 'strappo': 1, 'tempesta': 1, 'tirare': 1}

La sesta parola è: acqua
